# Дополнительный проект
В папке subsid `(shared/homeworks/python_ds_miniprojects/5_subsid)` находятся файлы `(tm_sales_1, tm_sales_2, ...)` с продажами продуктов через телемаркетинг. Каждый файл содержит, как минимум, 4 колонки (поля): FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.

Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе.

# Особенности данных:

1. сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить
2. поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны
3. продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)
4. если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается

Сохраните результат в датафрэйм с разделителем ;, содержащий корректные подключения.

Note: обратите внимание на то, как pandas переводит дату из строки, возможно вам понадобится параметр format

Пример содержания итогового файла (колонка difference это разница между START_DTTM и END_DTTM):

ACT_DTTM | FILIAL_ID |difference
---------|-----------|-----------
20-03-2020 14:59 | 1 | 255 days 09:01:00

In [1]:
import pandas as pd

In [147]:
tms1 = pd.read_csv('data/tm_sales_1.csv', sep=';')
tms2 = pd.read_csv('data/tm_sales_2.csv', sep=';')
tms3 = pd.read_csv('data/tm_sales_3.csv', sep=';')
log = pd.read_csv('data/prod_activations_logs.csv', sep=';', parse_dates=['START_DTTM', 'END_DTTM'], dayfirst=True)
tms1.head()

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25


In [63]:
tms = pd.concat([tms1, tms2, tms3], ignore_index=True)
tms2 = tms.copy()
tms2

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
4,NaN,4,7421,12-03-2020 11:25
5,8362218,7,9879,05-03-2020 11:42
6,2185490,2,3210,16-03-2020 16:28
7,id5764122,3,1499,18-03-2020 15:44
8,id7642700,6,3020,15-03-2020 14:21
9,id1374509,2,5677,17-03-2020 11:48


**1. Cотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить**

In [38]:
log[~(log.SUBS_ID.str.startswith('id'))]

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM


In [54]:
tms.SUBS_ID.apply(type)

0      <class 'str'>
1      <class 'str'>
2      <class 'str'>
3      <class 'str'>
4    <class 'float'>
5      <class 'str'>
6      <class 'str'>
7      <class 'str'>
8      <class 'str'>
9      <class 'str'>
Name: SUBS_ID, dtype: object

In [64]:
#заковыристо
mask = ~(tms.SUBS_ID.str.startswith('id').fillna(True))
x=tms.loc[mask, 'SUBS_ID']
x = 'id' + x

tms.update(x)
tms

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
4,NaN,4,7421,12-03-2020 11:25
5,id8362218,7,9879,05-03-2020 11:42
6,id2185490,2,3210,16-03-2020 16:28
7,id5764122,3,1499,18-03-2020 15:44
8,id7642700,6,3020,15-03-2020 14:21
9,id1374509,2,5677,17-03-2020 11:48


In [72]:
#решение проще
mask = ~(tms2.SUBS_ID.str.startswith('id').fillna(True))
tmsm = tms2[mask]

tms2.loc[mask, 'SUBS_ID'] = 'id' + tmsm['SUBS_ID']
tms2

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
4,NaN,4,7421,12-03-2020 11:25
5,id8362218,7,9879,05-03-2020 11:42
6,id2185490,2,3210,16-03-2020 16:28
7,id5764122,3,1499,18-03-2020 15:44
8,id7642700,6,3020,15-03-2020 14:21
9,id1374509,2,5677,17-03-2020 11:48


**2. Если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается**

In [10]:
tm_sales.shape

(9, 4)

In [13]:
tm_sales.loc[~tm_sales.SUBS_ID.str.startswith('id'),
             'SUBS_ID'] = 'id' + tm_sales.loc[~tm_sales.SUBS_ID.str.startswith('id'), 'SUBS_ID']
tm_sales

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
5,id8362218,7,9879,05-03-2020 11:42
6,id2185490,2,3210,16-03-2020 16:28
7,id5764122,3,1499,18-03-2020 15:44
8,id7642700,6,3020,15-03-2020 14:21
9,id1374509,2,5677,17-03-2020 11:48


In [15]:
sales_logs = prod_activation_logs.merge(tm_sales, on=['SUBS_ID', 'PROD_ID'])
sales_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM
0,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00,1,20-03-2020 14:59
1,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03,5,19-03-2020 13:00
2,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00,3,17-03-2020 10:17
3,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00,2,16-03-2020 16:28
4,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42,6,15-03-2020 14:21


In [16]:
sales_logs.dtypes

SUBS_ID       object
PROD_ID        int64
START_DTTM    object
END_DTTM      object
FILIAL_ID      int64
ACT_DTTM      object
dtype: object

In [18]:
sales_logs.START_DTTM = pd.to_datetime(sales_logs.START_DTTM)

In [19]:
sales_logs.END_DTTM = pd.to_datetime(sales_logs.END_DTTM, format='%d-%m-%Y %H:%M')

In [20]:
sales_logs.dtypes

SUBS_ID               object
PROD_ID                int64
START_DTTM    datetime64[ns]
END_DTTM      datetime64[ns]
FILIAL_ID              int64
ACT_DTTM              object
dtype: object

In [21]:
sales_logs.head()

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,20-03-2020 14:59
1,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,5,19-03-2020 13:00
2,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,17-03-2020 10:17
3,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,16-03-2020 16:28
4,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,15-03-2020 14:21


In [23]:
sales_logs['difference'] = sales_logs.END_DTTM - sales_logs.START_DTTM
sales_logs.head()

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM,difference
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,20-03-2020 14:59,255 days 09:01:00
1,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,5,19-03-2020 13:00,0 days 00:03:00
2,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,17-03-2020 10:17,8 days 00:43:00
3,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,16-03-2020 16:28,259 days 07:32:00
4,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,15-03-2020 14:21,0 days 09:21:00


In [25]:
result = sales_logs.loc[sales_logs.difference > pd.Timedelta(5, 'm')]
result

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM,difference
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,20-03-2020 14:59,255 days 09:01:00
2,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,17-03-2020 10:17,8 days 00:43:00
3,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,16-03-2020 16:28,259 days 07:32:00
4,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,15-03-2020 14:21,0 days 09:21:00


In [26]:
result.SUBS_ID.sort_values().values

array(['id2185490', 'id4651830', 'id5416547', 'id7642700'], dtype=object)

In [ ]:
result.to_csv('result.csv', index=False)

Ответ: `id2185490, id4651830, id5416547, id7642700`

In [75]:
tms = tms.dropna(subset=['SUBS_ID']).reset_index(drop=True)
tms

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
4,id8362218,7,9879,05-03-2020 11:42
5,id2185490,2,3210,16-03-2020 16:28
6,id5764122,3,1499,18-03-2020 15:44
7,id7642700,6,3020,15-03-2020 14:21
8,id1374509,2,5677,17-03-2020 11:48


In [99]:
log = log.reset_index(drop=True)
log

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00
1,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00
4,id8238421,1859,2020-03-01 11:42:00,2020-03-01 11:43:00
5,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00
6,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00
7,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00
8,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00


#3

In [156]:
import numpy as np

delta = log.END_DTTM - log.START_DTTM
mask = delta < np.timedelta64(5, 'm')
#mask = abs(delta.values.astype('timedelta64[m]').astype('int')) < 5

log = log.drop(log[mask].index).reset_index()
log

,index,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
0,1,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00
1,2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00
2,3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00
3,3,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00
4,2,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00
5,6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00
6,2,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00


#4

In [157]:
tms

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
4,id8362218,7,9879,05-03-2020 11:42
5,id2185490,2,3210,16-03-2020 16:28
6,id5764122,3,1499,18-03-2020 15:44
7,id7642700,6,3020,15-03-2020 14:21
8,id1374509,2,5677,17-03-2020 11:48


In [161]:
log.dtypes

index                  int64
SUBS_ID               object
PROD_ID               object
START_DTTM    datetime64[ns]
END_DTTM      datetime64[ns]
dtype: object

In [162]:
tms.dtypes

SUBS_ID      object
FILIAL_ID     int64
PROD_ID       int64
ACT_DTTM     object
dtype: object

In [163]:
log.PROD_ID = log.PROD_ID.astype('int64')
log.dtypes

index                  int64
SUBS_ID               object
PROD_ID                int64
START_DTTM    datetime64[ns]
END_DTTM      datetime64[ns]
dtype: object

In [16]:
res = pd.merge(log, tms, on=['SUBS_ID', 'PROD_ID'], how='inner')
res

,index,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM
0,0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,20-03-2020 14:59
1,3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,17-03-2020 10:17
2,6,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,16-03-2020 16:28
3,7,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,15-03-2020 14:21


In [17]:
res['difference'] = res.END_DTTM - res.START_DTTM
res

,index,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM,difference
0,0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,20-03-2020 14:59,255 days 09:01:00
1,3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,17-03-2020 10:17,8 days 00:43:00
2,6,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,16-03-2020 16:28,259 days 07:32:00
3,7,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,15-03-2020 14:21,0 days 09:21:00


In [19]:
res=res[['ACT_DTTM', 'FILIAL_ID', 'difference']]
res

,ACT_DTTM,FILIAL_ID,difference
0,20-03-2020 14:59,1,255 days 09:01:00
1,17-03-2020 10:17,3,8 days 00:43:00
2,16-03-2020 16:28,2,259 days 07:32:00
3,15-03-2020 14:21,6,0 days 09:21:00


In [20]:
res.to_csv('res.csv', index=False, sep=';')